In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [11]:
df=pd.read_csv("Churn_Modelling.csv")

In [12]:
## Preproccesing
## DOp irrelavent
data=df.drop(["RowNumber","CustomerId","Surname"],axis=1)

In [13]:
## ENcode Categorical
label_encoder_gender=LabelEncoder()
data["Gender"]=label_encoder_gender.fit_transform(data["Gender"])
data.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0


In [14]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encode_geo=OneHotEncoder()
geo_encoder=one_hot_encode_geo.fit_transform(data[["Geography"]])

In [15]:
one_hot_encode_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [16]:
geo_encode_df=pd.DataFrame(geo_encoder.toarray(),columns=one_hot_encode_geo.get_feature_names_out(['Geography']))

In [17]:
## combine all columns
data=pd.concat([data.drop("Geography",axis=1),geo_encode_df],axis=1)

In [19]:
## save the encoders and scalers
with open("label_encoder_gender.pkl",'wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('one_hot_encoder_geo.pkl',"wb") as file:
    pickle.dump(one_hot_encode_geo,file)

In [20]:
data.head(2)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0


In [21]:
### Divide the data into independent and dependent

X=data.drop("EstimatedSalary",axis=1)
y=data.EstimatedSalary

###split the data

X_train ,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## scale the features
scaler=StandardScaler()

X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [22]:
with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)

In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

2024-11-02 21:44:55.145105: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-02 21:44:55.472902: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-02 21:44:55.473001: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-02 21:44:55.529282: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-02 21:44:55.654107: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-02 21:44:55.655742: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [24]:
## BUild model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), #HL1
    Dense(32,activation="relu"), ## HL2
    Dense(1) # o/P layer
])

/home/raja/learnings/ANN/venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

# Set up TensorBoard
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [27]:
# Set up Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [28]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[early_stopping_callback, tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 101043.7031 - mae: 101043.7031 - val_loss: 98524.5078 - val_mae: 98524.5078
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 100017.1797 - mae: 100017.1797 - val_loss: 96922.3047 - val_mae: 96922.3047
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 97813.5859 - mae: 97813.5859 - val_loss: 92776.5469 - val_mae: 92776.5469
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 93464.9531 - mae: 93464.9531 - val_loss: 85899.0703 - val_mae: 85899.0703
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 86281.2969 - mae: 86281.2969 - val_loss: 77055.1094 - val_mae: 77055.1094
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 75286.3750 - mae: 75286.3750 - val_loss: 67935.1094 - val_mae: 67935.1094
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 67187.1875 - mae: 67187.1875 - val_loss: 60226.9844 - val_mae: 60226.9844
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/ste